In [1]:
import pandas as pd 
import numpy as np



In [12]:
csv_path_druckkopf= r"C:\Users\Vincent\Documents\Masterarbeit\Data\Prusa\final_csv_files\prusa_accelerometer_druckkopf.csv"

In [13]:
df_druckkopf = pd.read_csv(csv_path_druckkopf)
#df_druckbett = pd.read_csv(csv_path_druckbett)




In [14]:
df_druckkopf.columns

Index(['Acceleration_X', 'Acceleration_Y', 'Acceleration_Z', 'Timestamp',
       'Printer', 'PartName', 'SlicerSettings', 'Class'],
      dtype='object')

In [15]:
df_druckkopf_cls= {cls: df_druckkopf[df_druckkopf['Class'] == cls] for cls in df_druckkopf['Class'].unique()}

In [16]:
df_druckkopf_cls[2]["SlicerSettings"].unique()

array(['underextrusion4', 'underextrusion5'], dtype=object)

In [17]:
df_druckkopf_good=df_druckkopf_cls[0]
df_druckkopf_stringing=df_druckkopf_cls[1]
df_druckkopf_underextrusion=df_druckkopf_cls[2]


In [18]:
df_druckkopf_good.columns

Index(['Acceleration_X', 'Acceleration_Y', 'Acceleration_Z', 'Timestamp',
       'Printer', 'PartName', 'SlicerSettings', 'Class'],
      dtype='object')

In [19]:
# make features for the segments

from make_features_acceleration import butter_lowpass_filter,apply_fft_normalized
import sys

rms_features_good = pd.DataFrame()

segment_length = 500  # Define the length of each segment

columns_interest=['Acceleration_X', 'Acceleration_Y', 'Acceleration_Z']
    
# Calculate RMS for each segment in each column
for col in columns_interest:
    col_rms = []
    for start in range(0, len(df_druckkopf_good), segment_length):
        segment = df_druckkopf_good[col].iloc[start:start + segment_length]
        if len(segment) == segment_length:
            rms_value = np.sqrt(np.mean(segment**2))
            peak_frequency = np.max(segment)
            print(type(segment))
        
            col_rms.append(rms_value)

    
    # Add the RMS values as a new column in the rms_features dataframe
    rms_features_good[f'{col}_rms'] = col_rms
    rms_features_good["Class"]=0




<class 'pandas.core.series.Series'>


SystemExit: 

c:\Users\Vincent\miniconda3\envs\autoencoder_lightning\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
rms_features_good

,Acceleration_X_rms,Class,Acceleration_Y_rms,Acceleration_Z_rms
0,0.076395,0,0.940883,0.426050
1,0.063033,0,0.927070,0.447716
2,0.063902,0,0.936460,0.430346
3,0.062366,0,0.906985,0.485283
4,0.063845,0,0.924220,0.456897
...,...,...,...,...
16248,0.038376,0,0.895714,0.498755
16249,0.024194,0,0.905027,0.483967
16250,0.021649,0,0.930915,0.437558
16251,0.013151,0,0.904444,0.485283


In [ ]:
# peak frequency , peak amplitude 

from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq

def butter_lowpass_filter(data, cutoff, fs, order=5):
    
    # Calculate the Nyquist frequency
    nyquist = 0.5 * fs
    # Calculate the normal cutoff frequency needed for the filter , if normal_cutoff= 1 the nyquist frequency will be taken.
    normal_cutoff = cutoff / nyquist
    # Design the Butterworth filter
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    # Apply the Butterworth filter to the data
    y = filtfilt(b, a, data)
    return y


def apply_fft_normalized(data, fs):
    # Step 1: Calculate the number of samples in the data
    n = len(data)
    
    # Step 2: Compute the FFT of the data
    fft_data = np.fft.fft(data)
    
    # Step 3: Normalize the amplitude of the FFT output
    fft_amplitude = np.abs(fft_data) / n
    
    # Step 4: Compute the frequency bins corresponding to the FFT output
    freq = np.fft.fftfreq(n, 1/fs)
    
    # Step 5: Prepare the single-sided amplitude spectrum
    single_sided_amplitude = fft_amplitude[:n//2]
    single_sided_amplitude[1:] = 2 * single_sided_amplitude[1:]
    
    # Step 6: Prepare the corresponding frequency bins for the single-sided spectrum
    single_sided_freq = freq[:n//2]
    
    # Step 7: Return the single-sided amplitude and frequency arrays
    return single_sided_amplitude, single_sided_freq

In [14]:
df_druckkopf_good.columns

Index(['Acceleration_X', 'Acceleration_Y', 'Acceleration_Z', 'Timestamp',
       'Printer', 'PartName', 'SlicerSettings', 'Class'],
      dtype='object')

In [15]:
df_druckkopf_good["Timestamp"]

0           2024-06-24 15:29:34.003
1           2024-06-24 15:29:34.011
2           2024-06-24 15:29:34.013
3           2024-06-24 15:29:34.014
4           2024-06-24 15:29:34.016
                     ...           
16064552    2024-06-13 10:54:22.811
16064553    2024-06-13 10:54:22.813
16064554    2024-06-13 10:54:22.814
16064555    2024-06-13 10:54:22.815
16064556    2024-06-13 10:54:22.817
Name: Timestamp, Length: 14628494, dtype: object

## Calculate the sampling frequency

In [36]:
# Convert timestamps to datetime objects
df_druckkopf_good['Timestamp'] = pd.to_datetime(df_druckkopf_good['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

# Calculate time differences in seconds
time_diffs = df_druckkopf_good['Timestamp'].diff().dt.total_seconds()

time_diffs = time_diffs.dropna()

# Calculate the average time difference
avg_time_diff = time_diffs.mean()

# Calculate the frequency (samples per second)
frequency = 1 / avg_time_diff

print(frequency)l


C:\Users\Vincent\AppData\Local\Temp\ipykernel_16348\4145794481.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_druckkopf_good['Timestamp'] = pd.to_datetime(df_druckkopf_good['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')


-15.1290968723988


In [38]:
print(min(time_diffs), max(time_diffs))

-1292794.027 1297293.049


In [33]:
print(min(time_diffs), max(time_diffs))

0.001 0.008


In [34]:
avg_time_diff = time_diffs.mean()

In [35]:
avg_time_diff

0.00325

In [23]:
frequency

-15.129096872398769

In [29]:

import numpy as np
import matplotlib.pyplot as plt

# Create a sample DataFrame with your timestamp format
data = {
    'timestamp': [
        '2024-06-24 15:29:34.003',
        '2024-06-24 15:29:34.011',
        '2024-06-24 15:29:34.013',
        '2024-06-24 15:29:34.014',
        '2024-06-24 15:29:34.016'
    ]
}
df = pd.DataFrame(data)

# Convert timestamps to datetime objects
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

# Calculate time differences in seconds
time_diffs = df['timestamp'].diff().dt.total_seconds()

# Calculate the average time difference
avg_time_diff = time_diffs.mean()

# Calculate the frequency (samples per second)
frequency = 1 / avg_time_diff

In [30]:
avg_time_diff

0.00325